<a href="https://colab.research.google.com/github/hechen2020/19ma573HuixinChen/blob/master/src/hw_es_bsm_knock_in.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''======
run once, then comment it off, and run again
========'''
!git clone https://github.com/hechen2020/19ma573HuixinChen.git

Cloning into '19ma573HuixinChen'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 298 (delta 40), reused 0 (delta 0), pack-reused 176
Receiving objects: 100% (298/298), 1.18 MiB | 4.61 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [2]:
cd ./19ma573HuixinChen/src/

/content/19ma573HuixinChen/src


##hw_es_bsm_knock_in

###Goal - to apply ES to Knock-In option associated to geometric Brownian motion

In [0]:
from contract_v01 import VanillaOption
from sde_1d_v01 import Gbm_1d

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import array
import scipy.stats as ss
from scipy.stats.mstats import gmean

In [0]:
#define a function of BM path generator
def BM_gen(T1, T2, n): #para: start time, end time, and the mesh number
  t = np.linspace(T1, T2, num = n+1) #init mesh
  W = np.zeros(n+1) #init BM
  #Run (3.2)
  for i in range(n):
    W[i+1] = W[i] + 1./np.sqrt(n) * np.random.normal()
    
  return t, W

In [0]:
def bsm_knock_in_price(self,
                       strike,
                       barrier,
                       maturity,
                       num_step,
                       num_path):
  
  s0 = self.init_state
  sigma = self.vol_ratio
  r = self.drift_ratio
  
  K = strike
  H = barrier
  T = maturity
  m = num_step
  n = num_path
  h = T/m
  
  
  arr_kipayoff = array.array('f',[])
  for i in range(n):
    [t, S] = [BM_gen(0, T, m)[0], s0*np.exp((r-0.5*sigma**2)*h + sigma*BM_gen(0, T, m)[1])]
    #plt.plot(t, S);
    
    stock = S[m]
    minS = min(S)
    payoff = 1000*np.where(stock > K and minS < H, 1, 0)
    arr_kipayoff.append(payoff)
    price = np.mean(arr_kipayoff)*np.exp(-r*T)


  return price

Gbm_1d.bsm_knock_in_price = bsm_knock_in_price

In [0]:
gbm1 = Gbm_1d(init_state = 95, drift_ratio = 0.05, vol_ratio = 0.15)

K = 96.
H = 85.
T = 0.25
num_step = 50
num_paths = 1000
num_trials = 1000


In [8]:
arr_knock_in_price = array.array('f', [])
for i in range(num_trials):
  price = gbm1.bsm_knock_in_price(K, H, T, num_step, num_paths)
  arr_knock_in_price.append(price)

print(arr_knock_in_price)

array('f', [37.527957916259766, 38.515533447265625, 32.59006881713867, 31.602489471435547, 39.50311279296875, 43.45342254638672, 48.39131164550781, 50.3664665222168, 47.40373611450195, 32.59006881713867, 38.515533447265625, 41.478267669677734, 40.49068832397461, 45.4285774230957, 49.37889099121094, 35.552799224853516, 45.4285774230957, 40.49068832397461, 32.59006881713867, 44.441001892089844, 43.45342254638672, 44.441001892089844, 41.478267669677734, 46.41615676879883, 50.3664665222168, 42.46584701538086, 47.40373611450195, 32.59006881713867, 36.54037857055664, 42.46584701538086, 39.50311279296875, 38.515533447265625, 41.478267669677734, 35.552799224853516, 54.31678009033203, 56.291934967041016, 31.602489471435547, 41.478267669677734, 49.37889099121094, 45.4285774230957, 42.46584701538086, 42.46584701538086, 45.4285774230957, 42.46584701538086, 43.45342254638672, 39.50311279296875, 42.46584701538086, 51.35404586791992, 28.63975715637207, 35.552799224853516, 44.441001892089844, 43.45342

Mean

MSE

Confidence Interval

In [9]:
mean = np.mean(arr_knock_in_price)
var = np.mean((arr_knock_in_price-mean)**2) #using variance approximation
ci = ss.t.interval(0.95, len(arr_knock_in_price)-1, loc = mean, scale=np.sqrt(var))
print(mean)
print(var)
print(ci)

41.944405
39.8465
(29.55730378637243, 54.33150541772913)
